In [ ]:
# First, let's set up the environment and load the Kedro project context
%load_ext kedro.ipython
%reload_kedro

In [6]:

import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [ ]:
# Load the intermediate data from the Kedro catalog
catalog = context.catalog

In [ ]:
# Load the processed data
ads_data = catalog.load('intermediate_ads_data')
play_store_data = catalog.load('intermediate_play_store_data')

In [ ]:
# Create a Dash application
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Marketing Analytics Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Ad Performance Comparison', children=[
            dcc.Graph(id='ad-performance-comparison'),
            dcc.Dropdown(
                id='ad-bank-dropdown',
                options=[{'label': bank, 'value': bank} for bank in ads_data['bank'].unique()],
                multi=True,
                value=ads_data['bank'].unique().tolist()
            ),
            dcc.Slider(
                id='ad-time-slider',
                min=ads_data['ad_time'].min(),
                max=ads_data['ad_time'].max(),
                value=ads_data['ad_time'].max(),
                marks={str(time): str(time) for time in ads_data['ad_time'].unique()}
            )
        ]),
        dcc.Tab(label='Play Store Review Sentiment', children=[
            dcc.Graph(id='play-store-sentiment'),
            dcc.Dropdown(
                id='review-version-dropdown',
                options=[{'label': version, 'value': version} for version in play_store_data['version'].unique()],
                multi=True,
                value=play_store_data['version'].unique().tolist()
            )
        ]),
        dcc.Tab(label='Impact of Ads on App Download', children=[
            dcc.Graph(id='ad-impact-downloads'),
            dcc.Dropdown(
                id='ad-impact-bank-dropdown',
                options=[{'label': bank, 'value': bank} for bank in ads_data['bank'].unique()],
                multi=True,
                value=ads_data['bank'].unique().tolist()
            )
        ]),
        dcc.Tab(label='Impact of Ads on Telegram Subscriptions', children=[
            dcc.Graph(id='ad-impact-telegram'),
            dcc.Dropdown(
                id='telegram-bank-dropdown',
                options=[{'label': bank, 'value': bank} for bank in ads_data['bank'].unique()],
                multi=True,
                value=ads_data['bank'].unique().tolist()
            )
        ]),
    ])
])

@app.callback(
    Output('ad-performance-comparison', 'figure'),
    [Input('ad-bank-dropdown', 'value'),
     Input('ad-time-slider', 'value')]
)
def update_ad_performance(selected_banks, selected_time):
    filtered_data = ads_data[(ads_data['bank'].isin(selected_banks)) & (ads_data['ad_time'] <= selected_time)]
    fig = px.bar(filtered_data, x='bank', y='views', color='ad_type', barmode='group')
    return fig

@app.callback(
    Output('play-store-sentiment', 'figure'),
    [Input('review-version-dropdown', 'value')]
)
def update_play_store_sentiment(selected_versions):
    filtered_data = play_store_data[play_store_data['version'].isin(selected_versions)]
    fig = px.line(filtered_data, x='date', y='sentiment', color='version')
    return fig

@app.callback(
    Output('ad-impact-downloads', 'figure'),
    [Input('ad-impact-bank-dropdown', 'value')]
)
def update_ad_impact_downloads(selected_banks):
    filtered_data = ads_data[ads_data['bank'].isin(selected_banks)]
    fig = px.scatter(filtered_data, x='ad_time', y='app_downloads', color='bank', trendline='ols')
    return fig

@app.callback(
    Output('ad-impact-telegram', 'figure'),
    [Input('telegram-bank-dropdown', 'value')]
)
def update_ad_impact_telegram(selected_banks):
    filtered_data = ads_data[ads_data['bank'].isin(selected_banks)]
    fig = px.line(filtered_data, x='ad_time', y='telegram_subscriptions', color='bank')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)